# Exploring Ubuntu Data with SQL Queries

dataset:https://www.kaggle.com/datasets/rtatman/ubuntu-dialogue-corpus

# Ubuntu Dialogue Corpus Description
## Context:
Building dialogue systems is challenging in NLP, especially ensuring natural-feeling conversations with virtual agents. This corpus aids in this research by offering natural dialogues for system evaluation.

## Content:
The Ubuntu Dialogue Corpus comprises nearly 1M two-person conversations sourced from Ubuntu chat logs for technical support. Each convo averages 8 turns, with a minimum of 3, all in text format.

## Attributes:
- `Folder`: Source folder for each dialogue.
- `DialogueID`: Unique ID for each conversation.
- `Date`: Timestamp of each dialogue line.
- `From`: Sender of the message.
- `To`: Recipient of the message (blank for the first turn).
- `Text`: Dialogue text enclosed in double quotes.

## Acknowledgements:
Collected by Ryan Lowe, Nissan Pow, Iulian V. Serban, and Joelle Pineau, provided under Apache License, 2.0. Citation: SIGDial 2015.

## Inspiration:
This dataset can facilitate building Ubuntu-help chatbots, leveraging its wealth of real-world Ubuntu support conversations.






In [ ]:
import os
import zipfile

# Path to the ZIP file
zip_file_path = '/content/drive/MyDrive/archive.zip'

# Directory to extract the contents
extracted_dir = '/content/drive/MyDrive'

# Create the directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Walk through the extracted directory
for dirname, _, filenames in os.walk(extracted_dir):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/content/drive/MyDrive/archive.zip
/content/drive/MyDrive/toc.csv
/content/drive/MyDrive/Colab Notebooks/Vijay_Bommireddy_Assignment2 (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Vijay_Bommireddy_Assignment2.ipynb
/content/drive/MyDrive/Colab Notebooks/Vijay_Bommireddy_lab3.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled0.ipynb
/content/drive/MyDrive/Colab Notebooks/MEPS (8).ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled1.ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled (8)
/content/drive/MyDrive/Colab Notebooks/Untitled (7)
/content/drive/MyDrive/Colab Notebooks/Untitled (6)
/content/drive/MyDrive/Colab Notebooks/__notebook_source__.ipynb
/content/drive/MyDrive/Colab Notebooks/Movie_Recommendation_System (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled (5)
/content/drive/MyDrive/Colab Notebooks/Book_Recommendation_System (1).ipynb
/content/drive/MyDrive/Colab Notebooks/Untitled (4)
/content/drive/MyDrive/Colab Notebooks/Gourav_Vemula_Spark_Assignment_3 (

### Now let's create a connector for the SQLite3.

In [ ]:
import sqlite3
conn = sqlite3.connect('ubuntu_data.db')
c = conn.cursor()

In [ ]:
c.execute('''
CREATE TABLE dialogs (
    folder INTEGER,
    dialogueID TEXT,
    date TEXT,
    sender TEXT,
    receiver TEXT,
    msg TEXT);
''')

In [ ]:
c.execute('''
CREATE TABLE dialogs2 (
    folder INTEGER,
    dialogueID TEXT,
    date TEXT,
    sender TEXT,
    receiver TEXT,
    msg TEXT);
''')

Now let's use pandas method read_csv to read the source files and the method to_sql to export it to our SQL database. I have also renamed the columns to avoid issues with reserved words (such as TEXT).

In [ ]:
import pandas as pd
# load the data into a Pandas DataFrame
data1 = pd.read_csv('/content/drive/MyDrive/Ubuntu-dialogue-corpus/dialogueText.csv')

# rename columns
data1.rename(columns=
{
"from": "sender",
"to": "receiver",
"text": 'msg'
}, inplace=True)

# write the data to a sqlite table
data1.to_sql('dialogs', conn, if_exists='append', index=False)

1038324

***Now we don't need the Pandas table data1 anymore, let's clean it up - and confirm it is empty***

In [ ]:
data1.drop(data1.index, inplace=True)
print(data1)

Empty DataFrame
Columns: [folder, dialogueID, date, sender, receiver, msg]
Index: []


***The next line of code repeats the process for the second table data2***

In [ ]:
# load the data into a Pandas DataFrame
data2 = pd.read_csv('/content/drive/MyDrive/Ubuntu-dialogue-corpus/dialogueText_196.csv')

# rename bad name columns
data2.rename(columns=
{
"from": "sender",
"to": "receiver",
"text": 'msg'
}, inplace=True)

# write the data to a sqlite table
data2.to_sql('dialogs2', conn, if_exists='append', index=False)

#clears the Pandas table
data2.drop(data2.index, inplace=True)

With all the preparations done, let's write our first SQL query. For that we have to use the SQL connector we have created.

Let's check 10 of the entries to have an idea of what our database looks like.

In [ ]:
c.execute('''
SELECT *
FROM dialogs
LIMIT 10;
''').fetchall()

[(3,
  '126125.tsv',
  '2008-04-23T14:55:00.000Z',
  'bad_image',
  None,
  "Hello folks, please help me a bit with the following sentence: 'Order here your personal photos or videos.' - I think the only allowed version is 'Order your personal videos or photos here.', but I'm not sure, are you?"),
 (3,
  '126125.tsv',
  '2008-04-23T14:56:00.000Z',
  'bad_image',
  None,
  'Did I choose a bad channel? I ask because you seem to be dumb like windows user'),
 (3,
  '126125.tsv',
  '2008-04-23T14:57:00.000Z',
  'lordleemo',
  'bad_image',
  'the second sentence is better english   and we are not dumb'),
 (3,
  '64545.tsv',
  '2009-08-01T06:22:00.000Z',
  'mechtech',
  None,
  'Sock Puppe?t'),
 (3, '64545.tsv', '2009-08-01T06:22:00.000Z', 'mechtech', None, 'WTF?'),
 (3,
  '64545.tsv',
  '2009-08-01T06:22:00.000Z',
  'richardcavell',
  'mechtech',
  "it's a wikipedia term.  There is Prodigy, PRDIGY and prgidy and they're all the same guy"),
 (3,
  '98758.tsv',
  '2009-05-20T07:58:00.000Z',
  

*Let's verify the number of distinct messages, folders and dialog IDs:*

In [ ]:
#print the quantity of messages

print('Number of messages = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM dialogs;
''').fetchall()[0][0]
))

#print the quantity of folders

print('Number of folders = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT folder)
FROM dialogs;
''').fetchall()[0][0]
))

#print the quantity of dialog IDs

print('Number of dialog IDs = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT dialogueID)
FROM dialogs;
''').fetchall()[0][0]
))

Number of messages = 1038235
Number of folders = 1
Number of dialog IDs = 346108


**In the next exploration we select one user at random ActionParnisp1 and check the number of messages they have exchanged with each user different than None in each of the dialogs they appear**

In [ ]:
c.execute('''
SELECT sender, receiver, dialogueID, COUNT(msg)
FROM dialogs
WHERE receiver <> 'None' AND sender = 'ActionParsnip1'
GROUP BY dialogueID, sender, receiver
ORDER BY receiver ASC, COUNT(msg) DESC;
''').fetchall()

[('ActionParsnip1', 'Abracadabr4', '92449.tsv', 1),
 ('ActionParsnip1', 'Aijse', '64788.tsv', 1),
 ('ActionParsnip1', 'Amnesia', '66842.tsv', 1),
 ('ActionParsnip1', 'Avash', '100671.tsv', 1),
 ('ActionParsnip1', 'B10S', '81338.tsv', 1),
 ('ActionParsnip1', 'Benwa', '100467.tsv', 3),
 ('ActionParsnip1', 'Blizzerand', '100664.tsv', 1),
 ('ActionParsnip1', 'Bodsda', '100642.tsv', 1),
 ('ActionParsnip1', 'Boohbah_', '100514.tsv', 1),
 ('ActionParsnip1', 'Born2Live', '100490.tsv', 1),
 ('ActionParsnip1', 'Broken_Ubuntu', '100468.tsv', 2),
 ('ActionParsnip1', 'CK-TECH', '100578.tsv', 2),
 ('ActionParsnip1', 'Cantcme', '70146.tsv', 1),
 ('ActionParsnip1', 'Carstairs1', '100551.tsv', 1),
 ('ActionParsnip1', 'Crayboff', '100697.tsv', 2),
 ('ActionParsnip1', 'Crooper', '100689.tsv', 2),
 ('ActionParsnip1', 'DaHopi', '89265.tsv', 1),
 ('ActionParsnip1', 'DawnLight', '100535.tsv', 2),
 ('ActionParsnip1', 'Dayofswords', '100691.tsv', 2),
 ('ActionParsnip1', 'DingTo', '100500.tsv', 1),
 ('ActionPar

*Let's repeat the same exploration with the table dialogs2.*

In [ ]:
c.execute('''
SELECT *
FROM dialogs2
LIMIT 10;
''').fetchall()

[(301,
  '1.tsv',
  '2004-11-23T11:49:00.000Z',
  'stuNNed',
  None,
  'any ideas why java plugin takes so long to load?'),
 (301, '1.tsv', '2004-11-23T11:49:00.000Z', 'crimsun', 'stuNNed', 'java 1.4?'),
 (301, '1.tsv', '2004-11-23T11:49:00.000Z', 'stuNNed', 'crimsun', 'yes'),
 (301,
  '1.tsv',
  '2004-11-23T11:49:00.000Z',
  'crimsun',
  'stuNNed',
  'java 1.5 loads _much_ faster'),
 (301,
  '1.tsv',
  '2004-11-23T11:50:00.000Z',
  'stuNNed',
  'crimsun',
  'noneus: how can i get 1.5 is there a .deb somewhere?'),
 (301, '1.tsv', '2004-11-23T11:50:00.000Z', 'crimsun', 'stuNNed', 'not yet.'),
 (301,
  '1.tsv',
  '2004-11-23T11:50:00.000Z',
  'stuNNed',
  'crimsun',
  'noneus: is this blackdown or sun?'),
 (301,
  '1.tsv',
  '2004-11-23T11:50:00.000Z',
  'stuNNed',
  'crimsun',
  'did you install just the jre?'),
 (301,
  '1.tsv',
  '2004-11-23T11:51:00.000Z',
  'crimsun',
  'stuNNed',
  "I use IBM's 1.4.2"),
 (301,
  '1.tsv',
  '2004-11-23T11:51:00.000Z',
  'crimsun',
  'stuNNed',
  '(j

In [ ]:
#print the quantity of messages

print('Number of messages = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM dialogs2;
''').fetchall()[0][0]
))

#print the quantity of folders

print('Number of folders = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT folder)
FROM dialogs2;
''').fetchall()[0][0]
))

#print the quantity of dialog IDs

print('Number of dialog IDs = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT dialogueID)
FROM dialogs2;
''').fetchall()[0][0]
))

Number of messages = 9212043
Number of folders = 173
Number of dialog IDs = 346108


Interestingly enough, this second set of messages has more than one folder.

Let's verify if messages identified with a dialog ID are present in more than one folder.

In [ ]:
c.execute('''
SELECT COUNT (DISTINCT folder), dialogueID
FROM dialogs2
GROUP BY dialogueID
HAVING COUNT (DISTINCT folder) > 1
ORDER BY COUNT (DISTINCT folder) DESC;

''').fetchall()

[(173, '1.tsv'),
 (140, '2.tsv'),
 (124, '3.tsv'),
 (115, '4.tsv'),
 (110, '5.tsv'),
 (105, '6.tsv'),
 (100, '7.tsv'),
 (96, '8.tsv'),
 (93, '9.tsv'),
 (93, '10.tsv'),
 (90, '11.tsv'),
 (88, '12.tsv'),
 (85, '13.tsv'),
 (82, '14.tsv'),
 (81, '15.tsv'),
 (79, '16.tsv'),
 (75, '17.tsv'),
 (74, '19.tsv'),
 (74, '18.tsv'),
 (73, '23.tsv'),
 (73, '22.tsv'),
 (73, '21.tsv'),
 (72, '20.tsv'),
 (71, '24.tsv'),
 (68, '29.tsv'),
 (68, '28.tsv'),
 (68, '27.tsv'),
 (68, '26.tsv'),
 (68, '25.tsv'),
 (67, '32.tsv'),
 (67, '31.tsv'),
 (67, '30.tsv'),
 (65, '33.tsv'),
 (64, '34.tsv'),
 (63, '35.tsv'),
 (62, '36.tsv'),
 (61, '37.tsv'),
 (60, '39.tsv'),
 (59, '44.tsv'),
 (59, '43.tsv'),
 (59, '42.tsv'),
 (59, '41.tsv'),
 (59, '40.tsv'),
 (59, '38.tsv'),
 (58, '48.tsv'),
 (58, '47.tsv'),
 (58, '46.tsv'),
 (58, '45.tsv'),
 (57, '50.tsv'),
 (57, '49.tsv'),
 (55, '55.tsv'),
 (55, '54.tsv'),
 (55, '53.tsv'),
 (55, '52.tsv'),
 (55, '51.tsv'),
 (54, '57.tsv'),
 (54, '56.tsv'),
 (53, '58.tsv'),
 (52, '59.tsv'),

In the next code window we will merge both of the tables dialogs and dialogs2 into a third table alldialogs.

It was chosen to use an UNION to do this merging because, in the case there is repetition of values, we don't want to bring this repetition to the final result (and it would happen in case we used the INSERT INTO method for both tables). So we make an UNION and INSERT INTO an empty table.

For a detailed explanation on the different methods, refer to the site below:

In [ ]:
c.execute('''CREATE TABLE alldialogs (
    folder INTEGER,
    dialogueID TEXT,
    date TEXT,
    sender TEXT,
    receiver TEXT,
    msg TEXT);''')

c.execute('''
INSERT INTO alldialogs
SELECT *
FROM(
SELECT folder, dialogueID, date, sender, receiver, msg
    FROM dialogs2
UNION
SELECT folder, dialogueID, date, sender, receiver, msg
    FROM dialogs
)
''').fetchall()

[]

Let's do the same "measuring" of the resultant database:

In [ ]:
#print the quantity of messages

print('Number of messages = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM alldialogs;
''').fetchall()[0][0]
))

#print the quantity of folders

print('Number of folders = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT folder)
FROM alldialogs;
''').fetchall()[0][0]
))

#print the quantity of dialog IDs

print('Number of dialog IDs = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT dialogueID)
FROM alldialogs;
''').fetchall()[0][0]
))

Number of messages = 9187150
Number of folders = 173
Number of dialog IDs = 346108


But don't take my word for it! Let's use the INSERT INTO method and satisfy your Data Scientist curiosity.

We will create a dialogdummy table, make the measurements, and compare the results:

In [ ]:
c.execute('''
CREATE TABLE dialogdummy AS SELECT * FROM dialogs;
''').fetchall()

c.execute('''
INSERT INTO dialogdummy
SELECT *
FROM dialogs2;
''').fetchall()

[]

In [ ]:
#print the quantity of messages

print('Number of messages = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM dialogdummy;
''').fetchall()[0][0]
))

#print the quantity of folders

print('Number of folders = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT folder)
FROM dialogdummy;
''').fetchall()[0][0]
))

#print the quantity of dialog IDs

print('Number of dialog IDs = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT dialogueID)
FROM dialogdummy;
''').fetchall()[0][0]
))

Number of messages = 10250278
Number of folders = 173
Number of dialog IDs = 346108


You can see that the number of messages for dialogdummy table is greater than the number for alldialogs (10250300 > 9187170).

In [ ]:
#print the quantity of messages

print('Total number of messages in dialogdummy = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM dialogdummy;
''').fetchall()[0][0]
))

#print the quantity of messages

print('Total number of messages in alldialogs = {}'.format(
c.execute('''
SELECT COUNT (msg)
FROM alldialogs;
''').fetchall()[0][0]
))

#print the quantity of messages

print('Number of distinct messages in dialogdummy = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT msg)
FROM dialogdummy;
''').fetchall()[0][0]
))

#print the quantity of messages

print('Number of distinct messages in alldialogs  = {}'.format(
c.execute('''
SELECT COUNT (DISTINCT msg)
FROM alldialogs;
''').fetchall()[0][0]
))

Total number of messages in dialogdummy = 10250278
Total number of messages in alldialogs = 9187150
Number of distinct messages in dialogdummy = 7232451
Number of distinct messages in alldialogs  = 7232451


Even though the total number of messages is higher for the dialogdummy table (created by a pure INSERT INTO), the number of distinct messages is the same between both tables, suggesting that the INSERT INTO method passed repeated content.

Let's delete the dialogdummy table and move on.

In [ ]:
c.execute('''
DROP TABLE dialogdummy
''')

If you wanted to get rid of the first two tables, you would uncomment the code below an run it.

In [ ]:
#c.execute('''
#DROP TABLE dialogs
#''')

#c.execute('''
#DROP TABLE dialogs2
#''')

Suppose you ran all the queries you wanted to, merging and concatenating tables, grouping columns and etc. Now you want to export the resulting database back to an csv file. You can do it following the codes in the window below:

In [ ]:
clients = pd.read_sql('''
                      SELECT *
                      FROM alldialogs
                      ORDER BY dialogueID ASC;
                      ''', conn)

# Save the DataFrame to a CSV file
clients.to_csv('/content/results.csv', index=False)